# Analysis using FlowMachine directly

## Mobile Data Usage

In this worked example we assume the role of an analyst within the MNO who has been granted access to use FlowMachine directly without going through FlowAPI. Our aim is to investigate how the number of mobile data session (MDS) events varies with the time of day.

### Load FlowMachine and connect to FlowDB

We start by importing the FlowMachine library. We also import [folium](https://python-visualization.github.io/folium/) and the folium `HeatMapWithTime` plugin, which we will use later to to visualise the data.

In [1]:
import flowmachine
import folium
from folium.plugins import HeatMapWithTime

Next, we connect FlowMachine to FlowDB.

In [2]:
flowmachine.connect()

Logger created with level DEBUG


FlowMachine version: 0.4.3+245.gf6ef90bd.dirty
Flowdb running on: localhost:9999/flowdb (connecting user: analyst)


### Get MDS event counts

We create a `TotalLocationEvents` query to calculate the number of MDS events per cell tower, at hourly intervals over the first 6 days of 2016.

In [ ]:
data_events_query = flowmachine.features.TotalLocationEvents(
    start="2016-01-01",
    stop="2016-01-07",
    table="events.mds",
    level="versioned-cell",
    interval="hour",
    size=10,
)

Then we call the `get_dataframe` method to run this query and get the result as a `pandas` DataFrame.

In [4]:
data_events = data_events_query.get_dataframe()

Next, we sum over the six days to get total hourly counts per cell tower location.

In [5]:
events_per_hour = data_events.groupby(["lon", "lat", "hour"]).sum().reset_index()

### Visualise data events on a heatmap

We will use the folium `HeatMapWithTime` plugin to display a heatmap of our MDS event counts. This plugin expects location weights between 0 and 1, so we first scale our event counts by the maximum event count. We then transform our DataFrame into a list of lists of lists that `HeatMapWithTime` will accept as an input, and finally display our heatmap.

In [6]:
events_per_hour["weight"] = events_per_hour["total"]/(events_per_hour["total"].max())

hours = range(24)
data_for_heatmap = [events_per_hour[events_per_hour.hour==hour][["lat", "lon", "weight"]].values.tolist() for hour in hours]

In [7]:
m = folium.Map(location=[27.96834547, 85.960067737], zoom_start=7, tiles='cartodbpositron')

HeatMapWithTime(data=data_for_heatmap, index=[str(hour) for hour in hours], radius=0.2, scale_radius=True).add_to(m)

m
